In [ ]:
!pip install git+https://github.com/nmdl-mizo/interface_master.git@develop
!mkdir -p cif_files
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/Si_mp-149_conventional_standard.cif -o 'cif_files/Si_mp-149_conventional_standard.cif'
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/ZnO_mp-2133_conventional_standard.cif -o 'cif_files/ZnO_mp-2133_conventional_standard.cif'

  Cloning https://github.com/nmdl-mizo/interface_master.git (to revision develop) to /private/var/folders/km/tznxn_d162bgzpqpzp9wcjn00000gn/T/pip-req-build-45avigsg
  Running command git clone --filter=blob:none --quiet https://github.com/nmdl-mizo/interface_master.git /private/var/folders/km/tznxn_d162bgzpqpzp9wcjn00000gn/T/pip-req-build-45avigsg
^C
ERROR: Operation cancelled by user
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1059  100  1059    0     0   1624      0 --:--:-- --:--:-- --:--:--  1624
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

In [13]:
from interfacemaster.hetero_searching import hetero_searcher
from pymatgen.core.structure import Structure

In [14]:
#input substrate & film information
substrate_file, film_file = 'cif_files/Si_mp-149_conventional_standard.cif', \
'cif_files/ZnO_mp-2133_conventional_standard.cif'
substrate_structure, film_structure = list(map(lambda x:Structure.from_file(x), [substrate_file, film_file]))
substrate_name, film_name = 'Si', 'ZnO'
my_searcher = hetero_searcher(substrate_structure, film_structure, substrate_name, film_name)

Might be time consuming with high index limitation. Recommend to start by 1. 

In [15]:
#matching
my_searcher.matching(film_max_miller = 1, substrate_max_miller = 1)

57 non-identical matchings found


In [16]:
#generating, might consume some time to get rid of simpling scaling cases
my_searcher.generating(max_anum = 200, min_slab_length = 15)

no folder
20 interfaces generated, files saved in the dict Si_ZnO_interfaces


In [17]:
#write interface_info into json data
my_searcher.write_results('interface_info')

In [18]:
#you can check all the information for this interface
my_searcher.res_dict[1]

{'film_prim_hkl': array([0, 0, 1]),
 'film_prim_v1': array([2, 0, 0]),
 'film_prim_v2': array([ 0, -2,  0]),
 'film_conv_hkl': array([0, 0, 1]),
 'film_conv_v1': array(['2', '0', '0'], dtype='<U1'),
 'film_conv_v2': array(['0', '-2', '0'], dtype='<U2'),
 'substrate_prim_hkl': array([1, 1, 1]),
 'substrate_prim_v1': array([-2,  1,  1]),
 'substrate_prim_v2': array([-1,  2, -1]),
 'substrate_conv_hkl': array([-1, -1, -1]),
 'substrate_conv_v1': array(['1/2', '1/2', '-1'], dtype='<U3'),
 'substrate_conv_v2': array(['-1/2', '1', '-1/2'], dtype='<U4'),
 'CSL area': 38.85031,
 'strain': array([[ 1.01211998, -0.00605999, -0.00605999],
        [-0.00605999,  1.01211998, -0.00605999],
        [-0.00605999, -0.00605999,  1.01211998]]),
 'atom_num': 78,
 'substrate_prim_CNID_express': array([['0', '1/2'],
        ['-1/2', '0'],
        ['1/2', '-1/2']], dtype='<U4'),
 'film_prim_CNID_express': array([['1/3', '-2/3'],
        ['2/3', '-1/3'],
        ['0', '0']], dtype='<U4'),
 'substrate_conv_CNI

The searching results provides all the interfaces satsifying your requirement, you can get the interface object by calling its index.

In [19]:
selected_interface = my_searcher.interfaces[1]

Then you can customize the interface structure using this selected interface object

In [20]:
help(selected_interface.get_bicrystal)

Help on method get_bicrystal in module interfacemaster.interface_generator:

get_bicrystal(dydz=None, dx=0, dp1=0, dp2=0, xyz_1=None, xyz_2=None, vx=0, filename='POSCAR', two_D=False, filetype='VASP', mirror=False, KTI=False) method of interfacemaster.interface_generator.core instance
    generate a cif file for the bicrystal structure
    
    Parameters
    ----------
    dydz : numpy array
        translation vector in the interface
    dx : float
        translation normal to the interface
    dp1, dp2 : float
        termination of slab 1, 2
    xyz1, xyz2 : list
        expansion of slab 1, 2
    vx : float
        vacuum spacing, default 0
    filename : str
        filename, default 'POSCAR'
    two_D : bool
        whether a two CSL
    filetype : str
        filetype, 'VASP' or 'LAMMPS', default 'VASP'
    mirror : bool
        mirror, default False
    KTI : bool
        KTI, default False



In [21]:
#e.g increase interface displacing
from numpy import ceil
min_slab_length = 15
x1, x2 = int(ceil(15 / selected_interface.height_1)), int(ceil(15/selected_interface.height_2))
selected_interface.get_bicrystal(two_D = True, xyz_1 = [x1,1,1], xyz_2 = [x2,1,1], vx=5, dx=2, filename= 'POSCAR')

#Do high-throughput searching by displacing in CNID, prepare a INCAR & POTCAR file here :D

This will generate a series of VASP inputs folders



In [24]:
from numpy.linalg import norm
grid_dense = 1
n1 = int(ceil(norm(selected_interface.CNID[:,0])))
n2 = int(ceil(norm(selected_interface.CNID[:,1])))
print('CNID searching grid:')
print(n1, n2)

CNID searching grid:
2 2


In [25]:
# you can specify how many of atoms to be fixed at the two ending surface,
# here 33% of atoms at the ends will be fixed
min_slab_length = 15
x1, x2 = int(ceil(15 / selected_interface.height_1)), int(ceil(15/selected_interface.height_2))
selected_interface.sample_CNID(grid = [n1,n2] ,two_D = True, xyz_1 = [x1,1,1], xyz_2 = [x2,1,1], vx=5, dx=0, filename= 'POSCAR', fix_frac=0.33)

left bound: 0.1323297942146924
right bound: 0.7554828071862597
INCAR or POTCAR not found, please check
left bound: 0.1323297942146924
right bound: 0.7554828071862597
INCAR or POTCAR not found, please check
left bound: 0.1323297942146924
right bound: 0.7554828071862597
INCAR or POTCAR not found, please check
left bound: 0.1323297942146924
right bound: 0.7554828071862597
INCAR or POTCAR not found, please check
